# Assignment 2

**Author:** Tingjun Yuan

## Preparation

Import necessary libraries:

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os

## Obtain the Dataset

### Read the dataset from the files

**NOTE:** The following code assumes that `dataset.zip` from the assignment description page is unarchived as the directory `dataset`.

In [7]:
path_prefix = "dataset"

data_mapping = {}
for filename in os.listdir(path_prefix):
    scene_id = filename[:filename.find('.')]
    suffix = filename[filename.find('.')+1:]
    pair = data_mapping.setdefault(scene_id, {})
    match suffix:
        case 'edges':
            edges = pd.read_csv(
                os.path.join(path_prefix, filename),
                header=None,
                names=['target', 'source']
            )
            edges = edges[(edges['target'] != -1) & (edges['source'] != -1)]
            edges['target'] = edges['target'].astype(int)
            edges['source'] = edges['source'].astype(int)
            pair['edges'] = edges
        case 'nodes':
            nodes = pd.read_csv(
                os.path.join(path_prefix, filename),
                header=None,
                names=['node_id', 'current_x', 'current_y', 'previous_x',
                       'previous_y', 'future_x', 'future_y'],
                na_values='_'
            )
            nodes['node_id'] = nodes['node_id'].astype(int)
            pair['nodes'] = nodes
        case _:
            raise AssertionError(f'Unexpected suffix {suffix}')

In [8]:
data_mapping['13528908058']['edges']

,target,source
0,19585800,19590700
1,19585800,19595200
2,19585800,20000100
3,19590700,19595200
4,19590700,20000100
5,19591900,19594200
6,19591900,19595300
7,19591900,19595800
8,19592201,19595800
9,19592400,20000200


In [9]:
data_mapping['13528908058']['nodes']

,node_id,current_x,current_y,previous_x,previous_y,future_x,future_y
0,19502500,40050.0,-16544.0,40176.0,-16619.0,40205.0,-16357.0
1,19585800,16802.0,-11108.0,16140.0,-10573.0,17831.0,-11792.0
2,19590700,16846.0,-10526.0,16079.0,-9694.0,17528.0,-11131.0
3,19591900,11346.0,-6253.0,10833.0,-5840.0,12184.0,-6936.0
4,19592201,14232.0,-8556.0,13610.0,-7856.0,14867.0,-9359.0
5,19592400,5649.0,191.0,6542.0,-779.0,4809.0,1245.0
6,19592800,9097.0,1278.0,9323.0,1412.0,9221.0,1551.0
7,19594200,11262.0,-5387.0,10468.0,-4863.0,12387.0,-6358.0
8,19595200,18425.0,-11390.0,17495.0,-10254.0,20034.0,-12398.0
9,19595300,11060.0,-5800.0,10335.0,-5510.0,11890.0,-6411.0
